In [39]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

Load data from pickle file.

In [42]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat data as flat matrix, and use hot encoding for labels.

In [43]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


Use tensorflow to train multinomial logistic regression L2-regularized model with stochastid gradient descent.

In [75]:
#Regularization parameter
beta_reg = 0.1
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+beta_reg*tf.nn.l2_loss(weights)
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Train multinomial logistic regression with regularization.

In [76]:
num_steps = 8001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 318.620239
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.914355
Minibatch accuracy: 84.4%
Validation accuracy: 80.2%
Minibatch loss at step 1000: 1.055451
Minibatch accuracy: 76.6%
Validation accuracy: 79.2%
Minibatch loss at step 1500: 0.850208
Minibatch accuracy: 83.6%
Validation accuracy: 79.2%
Minibatch loss at step 2000: 0.951058
Minibatch accuracy: 85.9%
Validation accuracy: 78.9%
Minibatch loss at step 2500: 1.053545
Minibatch accuracy: 73.4%
Validation accuracy: 79.7%
Minibatch loss at step 3000: 1.067957
Minibatch accuracy: 78.9%
Validation accuracy: 79.4%
Minibatch loss at step 3500: 1.049423
Minibatch accuracy: 81.2%
Validation accuracy: 78.6%
Minibatch loss at step 4000: 0.973044
Minibatch accuracy: 82.8%
Validation accuracy: 80.2%
Minibatch loss at step 4500: 0.988494
Minibatch accuracy: 80.5%
Validation accuracy: 79.1%
Minibatch loss at step 5000: 0.932563
Minibatch accuracy: 85.2%
Validation accura

Create 1-layer hidden layer regularized NN with ReLu.

In [105]:
batch_size = 128
hidden_layer_size = 1024
beta_reg_nn = 0.0

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_hidden = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))
    
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)
    
    weights_output = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_output = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(hidden, weights_output)+biases_output

    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+beta_reg_nn*(tf.nn.l2_loss(weights_hidden)+tf.nn.l2_loss(weights_output))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    hidden_validation = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
    valid_prediction = tf.nn.softmax(
    tf.matmul(hidden_validation, weights_output) + biases_output)
    
    hidden_prediction = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_prediction, weights_output) + biases_output)

In [106]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #Uncomment the following two lines for extrelemy overfitting the data
    #step_overfit=1
    #offset = (step_overfit * batch_size) % (train_labels.shape[0] - batch_size)
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))



Initialized
Minibatch loss at step 0: 365.464294
Minibatch accuracy: 8.6%
Validation accuracy: 28.8%
Minibatch loss at step 1000: 13.224687
Minibatch accuracy: 78.9%
Validation accuracy: 82.9%
Minibatch loss at step 2000: 4.379684
Minibatch accuracy: 85.2%
Validation accuracy: 82.6%
Minibatch loss at step 3000: 3.454924
Minibatch accuracy: 83.6%
Validation accuracy: 82.4%
Minibatch loss at step 4000: 6.137101
Minibatch accuracy: 85.2%
Validation accuracy: 82.9%
Minibatch loss at step 5000: 4.941701
Minibatch accuracy: 86.7%
Validation accuracy: 80.5%
Minibatch loss at step 6000: 2.463027
Minibatch accuracy: 80.5%
Validation accuracy: 83.6%
Minibatch loss at step 7000: 5.042780
Minibatch accuracy: 83.6%
Validation accuracy: 84.2%
Minibatch loss at step 8000: 4.478261
Minibatch accuracy: 80.5%
Validation accuracy: 82.9%
Minibatch loss at step 9000: 5.584070
Minibatch accuracy: 82.8%
Validation accuracy: 82.2%
Minibatch loss at step 10000: 2.606863
Minibatch accuracy: 84.4%
Validation acc

Create 1-layer hidden layer NN with ReLu and dropout.

In [128]:
batch_size = 128
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_hidden = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))
    
    keep_prob = tf.placeholder(tf.float32)
    weights_hidden_dropped = tf.nn.dropout(weights_hidden, keep_prob)
    
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden_dropped) + biases_hidden)
    #hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)
    
    #Create a placeholder for dropout probability
    weights_output = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_output = tf.Variable(tf.zeros([num_labels]))
    
    weights_output_dropout = tf.nn.dropout(weights_output, keep_prob)
    
    logits = tf.matmul(hidden, weights_output_dropout)+biases_output
    #logits = tf.matmul(hidden, weights_output)+biases_output

    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    hidden_validation = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden)
    valid_prediction = tf.nn.softmax(
    tf.matmul(hidden_validation, weights_output) + biases_output)
    
    hidden_prediction = tf.nn.relu(tf.matmul(tf_test_dataset, weights_hidden) + biases_hidden)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_prediction, weights_output) + biases_output)

In [129]:
num_steps = 10001
dropout_parameter = 0.5

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #Uncomment the following two lines for extrelemy overfitting the data
    step_overfit=1
    offset = (step_overfit * batch_size) % (train_labels.shape[0] - batch_size)
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: dropout_parameter}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))



Initialized
Minibatch loss at step 0: 579.659302
Minibatch accuracy: 10.2%
Validation accuracy: 22.2%
Minibatch loss at step 1000: 1.596209
Minibatch accuracy: 99.2%
Validation accuracy: 69.9%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.7%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Minibatch loss at step 4000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.6%
Minibatch loss at step 5000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.0%
Minibatch loss at step 6000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 69.9%
Minibatch loss at step 7000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.3%
Minibatch loss at step 8000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 70.1%
Minibatch loss at step 9000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 69.9%
Minibatch loss at step 10000: 0.000000
Minibatch accuracy: 100.0%
Valid